<a href="https://colab.research.google.com/github/kendzisah/DeepImageClassifier/blob/main/Deep_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Depencies and setting up Computer

In [2]:
pip install tensorflow opencv-python matplotlib

In [26]:
import tensorflow as tf
import os

In [27]:
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

Preparing/Cleaning Data

In [33]:
import cv2
import imghdr
from matplotlib import pyplot as plt

In [29]:
data_dir = 'data'

In [30]:
image_exts = ['jpeg', 'jpg', 'bmp', 'png']

In [31]:
for image_class in os.listdir(data_dir):
  for image in os.listdir(os.path.join(data_dir, image_class)):
    image_path = os.path.join(data_dir, image_class, image)
    try:
      img = cv2.imread(image_path)
      tip = imghdr.what(image_path)
      if tip not in image_exts:
        print('Image not in ext list {}'.format(image_path))
        os.remove(image_path)
    except Exception as e:
      print('Issue with image{}'.format(image_path))

Image not in ext list data/happy-images/smiling-young-african-american-woman-260nw-2290057229.jpg
Image not in ext list data/happy-images/traitshappypeople.jpg
Image not in ext list data/happy-images/Happy20People.jpg
Image not in ext list data/happy-images/depositphotos_391577690-stock-photo-cheerful-woman-outstretched-hands-sitting.jpg
Image not in ext list data/sad-images/unpleasant-pain-sad-unhappy-handsome-600nw-1060449161.jpg
Image not in ext list data/sad-images/Sad-man-sitting-in-bedroom-thumbnail-732x549.jpg
Image not in ext list data/sad-images/portrait-sad-man-260nw-126009806.jpg
Image not in ext list data/sad-images/sad-person-pictures-2bnso9uiwlhrikrx.jpg
Image not in ext list data/sad-images/sad-person-pictures-1920-x-1200-qq00l14kkjed5hew.jpg
Image not in ext list data/sad-images/depositphotos_4584891-stock-photo-sad-man-in-a-empty.jpg
Image not in ext list data/sad-images/CC_HE_1221887081_SituationalDepression.jpg
Image not in ext list data/sad-images/depositphotos_1956

Loading Data